# A08 - Bootstrapping

In [1]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
from scipy import stats 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.linear_model import Ridge, ElasticNet, Lasso, LinearRegression, LogisticRegression
from sklearn.model_selection import LeaveOneOut, train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from scipy.stats import norm
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform, uniform  
from sklearn.utils import resample

## Advertising

In [2]:
df_a=pd.read_csv('Advertising.csv')
df_a.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [3]:
df_a.drop(columns=['Unnamed: 0'],inplace=True)
df_a.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [4]:
X = df_a.drop(columns=['sales'])
y=df_a['sales']
scaler=StandardScaler().fit(X)
X =scaler.fit_transform(X)
x=np.ones((len(X),1))
X_e=np.hstack((x,X))
ols=sm.OLS(y,X_e)
results=ols.fit()
print(results.params)


const    14.022500
x1        3.919254
x2        2.792063
x3       -0.022539
dtype: float64


In [5]:
intervalos= results.conf_int(alpha=0.05)
print(intervalos)

               0          1
const  13.787453  14.257547
x1      3.683666   4.154842
x2      2.540557   3.043569
x3     -0.274070   0.228993


### Bootstrap

In [6]:
n_iterations = 1000
n_size = len(X)  
coefs_bootstrap = []

X_con_constante = sm.add_constant(X) 

for i in range(n_iterations):
    X_res, y_res = resample(X_con_constante, y, replace=True, random_state=i)

    model = sm.OLS(y_res, X_res).fit()

    coefs_bootstrap.append(model.params)

medias_bootstrap = np.mean(coefs_bootstrap, axis=0)
std_bootstrap = np.std(coefs_bootstrap, axis=0)

print(f'La media de los coeficientes con bootstap es de: {medias_bootstrap}')
print(f'La desviacion estandar de los coeficientes con bootstrap es de: {std_bootstrap}')

La media de los coeficientes con bootstap es de: [ 1.40272717e+01  3.91026679e+00  2.78639359e+00 -1.18173851e-02]
La desviacion estandar de los coeficientes con bootstrap es de: [0.11978361 0.16329602 0.16215863 0.13668423]


In [15]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

search_spaces = {
    'alpha': Real(1e-2, 100, prior='log-uniform')
}
bayes_search = BayesSearchCV(
    estimator=Ridge(),
    search_spaces=search_spaces,
    n_iter=50,              
    cv=10,                   
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42
)

bayes_search.fit(X_scaled, y)

best_model = bayes_search.best_estimator_
best_alpha = bayes_search.best_params_['alpha']

coefs_base = [best_model.intercept_] + list(best_model.coef_.ravel())
medias_ridge = np.mean(coefs_base, axis=0)
std_ridge = np.std(coefs_base, axis=0)

print(f'mejor alpha {best_alpha}')
print(f'La media de los coeficientes con ridge es de: {medias_bootstrap}')
print(f'La desviacion estandar de los coeficientes con ridge es de: {std_bootstrap}')

n_iterations = 1000
coefs_ridge_bootstrap = []
model_ridge = Ridge(alpha=best_alpha)


for i in range(n_iterations):
    X_res, y_res = resample(X_scaled, y, replace=True, random_state=i)

    model_ridge.fit(X_res, y_res)

    iter_coefs = [model_ridge.intercept_] + list(model_ridge.coef_.ravel())
    coefs_ridge_bootstrap.append(iter_coefs)

coefs_ridge_bootstrap = np.array(coefs_ridge_bootstrap)
medias_ridge = np.mean(coefs_ridge_bootstrap, axis=0)
std_ridge = np.std(coefs_ridge_bootstrap, axis=0)

print(f'La media de los coeficientes con ridge y bootstarp es de: {medias_bootstrap}')
print(f'La desviacion estandar de los coeficientes con ridge y bootstarp es de: {std_bootstrap}')

mejor alpha 1.1687292343497746
La media de los coeficientes con ridge es de: [ 1.40272717e+01  3.91026679e+00  2.78639359e+00 -1.18173851e-02]
La desviacion estandar de los coeficientes con ridge es de: [0.11978361 0.16329602 0.16215863 0.13668423]
La media de los coeficientes con ridge y bootstarp es de: [ 1.40272717e+01  3.91026679e+00  2.78639359e+00 -1.18173851e-02]
La desviacion estandar de los coeficientes con ridge y bootstarp es de: [0.11978361 0.16329602 0.16215863 0.13668423]


## Default


In [ ]:
df_d=pd.read_csv('Default.csv')   
df_d.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138950
3,No,No,529.250605,35704.493940
4,No,No,785.655883,38463.495880
